# Dependance of Transmission vs H2O xairmass

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/11/24
- last update : 2024/12/03

This emulator is based from datagrid of atmospheric transparencies extracted from libradtran

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (10, 6),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Imports

In [ ]:
from getObsAtmo.getObsAtmo import ObsAtmo,validateObsName,Dict_Of_sitesPressures,getObsSiteDataFrame

# Check information

In [ ]:
Dict_Of_sitesPressures

### Choose Rubin-LSST observation sites but other sites may be chosen

In [ ]:
obssitename = 'LSST'
obs_str = validateObsName(obssitename) 
std_pressure = Dict_Of_sitesPressures[obs_str]

In [ ]:
emul =  ObsAtmo(obs_str= obssitename)

# Typical transmission plot

- show a typical transmission plot

In [ ]:
emul.plot_transmission()

# Setting atmospheric parameters

In [ ]:
WL = emul.GetWL()
airmasses = emul.AIRMASS
NAM = len(airmasses)
pwvs = emul.PWV
NPWV = len(pwvs)
ozs = emul.OZ
NOZ = len(ozs)

In [ ]:
hls_colors = sns.color_palette("hls", NAM)
sns.palplot(hls_colors)

## Check emulator with PWV absorption

Expect no difference

In [ ]:
hls_colors = sns.color_palette("hls", NPWV)
sns.palplot(hls_colors)

In [ ]:
am =1

fig, ax = plt.subplots()
for idx,pwv in enumerate(pwvs):

    transm = emul.GetPWVabsTransparencyArray(WL,am,pwv)
  
    if idx == 0: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
    else: 
        ax.plot(WL,transm,"-",color=hls_colors[idx])
        
title = f"ObsAtmo : H2O absorption for different pressures, am = {am:.1f}"
ax.set_title(title)
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("transmission")
ax.set_xlim(700.,1000.)
# place a text box in upper left in axes coords
#ax.text(0.5, 0.05, textboxstr, transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=textboxprops)
ax.grid()
ax.legend()

ax.axvline(719.0,color="k")
ax.axvline(725.0,color="k")
ax.axvline(728.0,color="k")
ax.axvline(817.0,color="k")
ax.axvline(823.0,color="k")
ax.axvline(828.0,color="k")
ax.axvline(900.0,color="k")
ax.axvline(910.0,color="k")
ax.axvline(915.0,color="k")
ax.axvline(935.0,color="k")
ax.axvline(952.0,color="k")
ax.axvline(975.0,color="k")

plt.show()

## Predefine a series of wavelengths

In [ ]:
WLtest = np.array([719.,725.,728.,817.,823.,828.,900.,910.,915.,935.,952.,975.])
NWLtest = len(WLtest)
#hls_colors = sns.color_palette("Spectral_r", NWLtest)
hls_colors = sns.color_palette("coolwarm",NWLtest)
#sns.palplot(hls_colors)

In [ ]:
binnumber = np.digitize(WLtest,WLtest)
binnumber

### Pick a palette from 700 to 1000 nm

In [ ]:
#WLrange = np.arange(700.,1001.,1)
#NWLrange = len(WLrange)
#hls_colors = sns.color_palette("Spectral_r", NWLrange)
#hls_colors = sns.color_palette("coolwarm",NWLrange)
#sns.palplot(hls_colors)
#binnumber = np.digitize(WLtest,WLrange)
#binnumber

## Different values of PWV0 and different airmasses

In [ ]:
#all_pwv0 = np.arange(0.1,12,2)
all_pwv0 = np.logspace(-1,1.15,12)
all_pwv0

In [ ]:
linestyles = ["-","--","-.",":",  "-","--","-.",":", "-","--","-.",":"]
linewidths = [4,4,4,4,3,3,3,3,2,2,2,2]

## Plot

### Plot linear scale

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    #norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = airmasses*pwv0
        y = transmarray[:,idx_wl]
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#ax.legend(loc='upper right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$T_{PWV}$")  
ax.set_title("PWV Air transmission vs $airmass \\times pwv$")

### Extinction in Y - lin scale, X - lin scale

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = airmasses*pwv0
        y = - 2.5*np.log10(transmarray[:,idx_wl])
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$")

### Extinction in Y - lin scale, X - log scale

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = np.log10(airmasses*pwv0)
        y = -2.5*np.log10(transmarray[:,idx_wl])
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$\\log_{10}(airmass \\times pwv$ (mm))")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$")

### Extinction in Y - log scale, X - log scale

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = np.log10(airmasses*pwv0)
        y = -2.5*np.log10(transmarray[:,idx_wl])
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$\\log_{10}(airmass \\times pwv$ (mm))")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$")
ax.set_yscale("log")

plt.show()

## Compare with band

In [ ]:
df = pd.read_csv("extinctioninbands_pwvxairmass.csv",index_col=0)

In [ ]:
df

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = airmasses*pwv0
        y = - 2.5*np.log10(transmarray[:,idx_wl])
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#df.plot.scatter(x="amxpwv", y="d_adu_0u_mag",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
#df.plot.scatter(x="amxpwv", y="d_adu_0g_mag",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
#df.plot.scatter(x="amxpwv", y="d_adu_0r_mag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df.plot.scatter(x="amxpwv", y="d_adu_0i_mag",ax=ax,marker="+",s=20,grid=True,c='brown',label="band LSST-I") 
df.plot.scatter(x="amxpwv", y="d_adu_0z_mag",ax=ax,marker="*",s=20,grid=True,c='green',label="band LSST-Z") 
df.plot.scatter(x="amxpwv", y="d_adu_0y_mag",ax=ax,marker="o",s=20,grid=True,c='purple',label="band LSST-Y") 



#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$")


plt.show()

In [ ]:
fig,ax = plt.subplots(1,1)

# external loop on PWV0 (linestyles and width)

for idx_pwv,pwv0 in enumerate(all_pwv0):

    # compute a new transmission array in airmass x wl
    transmarray = np.zeros((NAM,NWLtest))
    
    for idx_am,airmass in enumerate(airmasses):
        transmarray[idx_am,:]  = emul.GetPWVabsTransparencyArray(WLtest,airmass,pwv0)
    # normalize wrt airmass = 1 for each wavelength
    norm_am0 = np.copy(transmarray[0,:])
    
    # renormalize to first airmass
    #for idx_am,airmass in enumerate(airmasses):
    #    transmarray[idx_am,:] =  transmarray[idx_am,:]/norm_am0

    # plot attenuation 
    for idx_wl,wl in enumerate(WLtest):
        x = airmasses*pwv0
        y = - 2.5*np.log10(transmarray[:,idx_wl])
        
        bin_number_for_wl = np.digitize(wl,WLtest)-1
        the_wl_color = hls_colors[bin_number_for_wl]
        ls = linestyles[idx_pwv]
        lw = linewidths[idx_pwv]

        if idx_pwv==0:
            label = "$\\lambda = $" + f"{wl:.0f} nm"
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw,label=label)
        else:
            ax.plot(x,y,color=the_wl_color,ls = ls,lw=lw)

#df.plot.scatter(x="amxpwv", y="d_adu_0u_mag",ax=ax,marker=".",grid=True,c='b',label="band LSST-U") 
#df.plot.scatter(x="amxpwv", y="d_adu_0g_mag",ax=ax,marker=".",grid=True,c='g',label="band LSST-G") 
#df.plot.scatter(x="amxpwv", y="d_adu_0r_mag",ax=ax,marker=".",grid=True,c='r',label="band LSST-R") 
df.plot.scatter(x="amxpwv", y="d_adu_0i_mag",ax=ax,marker="+",s=20,grid=True,c='brown',label="band LSST-I") 
df.plot.scatter(x="amxpwv", y="d_adu_0z_mag",ax=ax,marker="*",s=20,grid=True,c='green',label="band LSST-Z") 
df.plot.scatter(x="amxpwv", y="d_adu_0y_mag",ax=ax,marker="o",s=20,grid=True,c='purple',label="band LSST-Y") 



#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$")
ax.set_yscale("log")
ax.set_xscale("log")
plt.show()

## Fit Lambert function

In [ ]:
from scipy.special import wrightomega, lambertw
from scipy.optimize import curve_fit

def absorption_lambertw(x, a, b ):
    return np.real((a * lambertw(x*b ,0)))

In [ ]:
fig,ax = plt.subplots(1,1)

xfit = np.linspace(0.,40.,100)

# loop on wavelength
for idx_wl,wl in enumerate(WLtest):

    all_x = []
    all_y = []

    # loop on pwv values
    for idx_pwv,pwv0 in enumerate(all_pwv0):

        # compute the transmission for all airmasses
        transmarray = np.zeros(NAM)
    
        for idx_am,airmass in enumerate(airmasses):
            transmarray[idx_am]  = emul.GetPWVabsTransparencyArray([wl],airmass,pwv0)

        
        x = airmasses*pwv0
        y = - 2.5*np.log10(transmarray)
        
        all_x.append(x)
        all_y.append(y)

    # concatenate for all pwv values
    x = np.concatenate(all_x)
    y = np.concatenate(all_y)


    # Do the fit
    try:
        popt, pcov = curve_fit(absorption_lambertw, x, y,  bounds=(0, [3., 2.]), method = 'trf')
        yfit = absorption_lambertw(xfit,popt[0], popt[1] )
        ax.plot(xfit,yfit,':',color="grey")
    except:
        pass

    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    label = "$\\lambda = $" + f"{wl:.0f} nm"

    index_sorted = np.argsort(x)
    
    ax.plot(x[index_sorted],y[index_sorted],color=the_wl_color,ls = ls,lw=lw,label=label)

df.plot.scatter(x="amxpwv", y="d_adu_0i_mag",ax=ax,marker="+",s=20,grid=True,c='brown',label="band LSST-I") 
df.plot.scatter(x="amxpwv", y="d_adu_0z_mag",ax=ax,marker="*",s=20,grid=True,c='green',label="band LSST-Z") 
df.plot.scatter(x="amxpwv", y="d_adu_0y_mag",ax=ax,marker="o",s=20,grid=True,c='purple',label="band LSST-Y") 

#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$ : Fit of Lambert W function")

plt.show()


In [ ]:
fig,ax = plt.subplots(1,1)

xfit = np.linspace(0.01,100.,100)

# loop on wavelength
for idx_wl,wl in enumerate(WLtest):

    all_x = []
    all_y = []

    # loop on pwv values
    for idx_pwv,pwv0 in enumerate(all_pwv0):

        # compute the transmission for all airmasses
        transmarray = np.zeros(NAM)
    
        for idx_am,airmass in enumerate(airmasses):
            transmarray[idx_am]  = emul.GetPWVabsTransparencyArray([wl],airmass,pwv0)

        
        x = airmasses*pwv0
        y = - 2.5*np.log10(transmarray)
        
        all_x.append(x)
        all_y.append(y)

    # concatenate for all pwv values
    x = np.concatenate(all_x)
    y = np.concatenate(all_y)


    # Do the fit
    try:
        popt, pcov = curve_fit(absorption_lambertw, x, y,  bounds=(0, [3., 2.]), method = 'trf')
        yfit = absorption_lambertw(xfit,popt[0], popt[1] )
        ax.plot(xfit,yfit,':',color="grey")
    except:
        pass

    bin_number_for_wl = np.digitize(wl,WLtest)-1
    the_wl_color = hls_colors[bin_number_for_wl]
    ls = linestyles[0]
    lw = linewidths[0]

    label = "$\\lambda = $" + f"{wl:.0f} nm"

    index_sorted = np.argsort(x)
    
    ax.plot(x[index_sorted],y[index_sorted],color=the_wl_color,ls = ls,lw=lw,label=label)

df.plot.scatter(x="amxpwv", y="d_adu_0i_mag",ax=ax,marker="+",s=20,grid=True,c='brown',label="band LSST-I") 
df.plot.scatter(x="amxpwv", y="d_adu_0z_mag",ax=ax,marker="*",s=20,grid=True,c='green',label="band LSST-Z") 
df.plot.scatter(x="amxpwv", y="d_adu_0y_mag",ax=ax,marker="o",s=20,grid=True,c='purple',label="band LSST-Y") 

#ax.legend(loc='lower right')
ax.legend(bbox_to_anchor=(1.05, 1.01))
ax.set_xlabel("$airmass \\times pwv$ (mm)")  
ax.set_ylabel("$-2.5 \\log_{10}(T_{PWV})$ (mag)")  
ax.set_title("PWV Air extinction vs $airmass \\times pwv$ : Fit of Lambert W function")
ax.set_xscale("log")
ax.set_yscale("log")
plt.show()
